In [1]:
 !pip install transformers

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
pip install openpyxl

     |████████████████████████████████| 242 kB 888 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [5]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [6]:
pip install ktrain

     |████████████████████████████████| 25.3 MB 883 kB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 22.3 MB 39.1 MB/s            
     |████████████████████████████████| 981 kB 46.3 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 263 kB 50.8 MB/s            
     |████████████████████████████████| 2.8 MB 48.3 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 468 kB 57.9 MB/s            
     |████████████████████████████████| 3.3 MB 54.7 MB/s            
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for ktrain: filename=ktrain-0.30.0-py3-none-any.wh

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [8]:
import ktrain
from ktrain import text

In [9]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Token'].tolist()
y_train=train['Label'].tolist()
x_validation=validation['Token'].tolist()
y_validation=validation['Label'].tolist()
x_test=test['Token'].tolist()
y_test=test['Label'].tolist()

In [10]:
x_train= np.array(x_train)
x_validation= np.array(x_validation)
x_test= np.array(x_test)


In [11]:
categories = ['sad', 'angry','surprise','happy','fear','disgust']

In [12]:
model_name = 'bert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [13]:
train = trans.preprocess_train(x_train,y_train)
valid = trans.preprocess_test(x_validation,y_validation)

preprocessing train...
language: bn
train sequence lengths:
	mean : 26
	95percentile : 74
	99percentile : 173


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 26
	95percentile : 76
	99percentile : 165


In [14]:
model = trans.get_classifier()

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

In [15]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [16]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
weight={i:class_weight[i] for i in range(6)}

In [17]:
learner.fit_onecycle(2e-5,20,class_weight=weight) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/20
427/427 [==============================] - 82s 145ms/step - loss: 1.7829 - accuracy: 0.1805 - val_loss: 1.7144 - val_accuracy: 0.2137
Epoch 2/20
427/427 [==============================] - 58s 137ms/step - loss: 1.7464 - accuracy: 0.2367 - val_loss: 1.6451 - val_accuracy: 0.3541
Epoch 3/20
427/427 [==============================] - 58s 136ms/step - loss: 1.6885 - accuracy: 0.3130 - val_loss: 1.5955 - val_accuracy: 0.3479
Epoch 4/20
427/427 [==============================] - 58s 136ms/step - loss: 1.6198 - accuracy: 0.3482 - val_loss: 1.5549 - val_accuracy: 0.3463
Epoch 5/20
427/427 [==============================] - 59s 138ms/step - loss: 1.5592 - accuracy: 0.3970 - val_loss: 1.5802 - val_accuracy: 0.3339
Epoch 6/20
427/427 [==============================] - 58s 137ms/step - loss: 1.4950 - accuracy: 0.4284 - val_loss: 1.5168 - val_accuracy: 0.3963
Epoch 7/20
427/427 [==============================] - 58s 135ms/ste

In [18]:
learner.validate(class_names=categories)

              precision    recall  f1-score   support

         sad       0.50      0.58      0.54       137
       angry       0.10      0.10      0.10        72
    surprise       0.11      0.10      0.10        30
       happy       0.66      0.64      0.65       195
        fear       0.36      0.37      0.36       132
     disgust       0.33      0.27      0.30        75

    accuracy                           0.44       641
   macro avg       0.34      0.34      0.34       641
weighted avg       0.44      0.44      0.44       641



array([[ 80,  16,   3,   9,  17,  12],
       [ 25,   7,   4,  15,  16,   5],
       [  4,   4,   3,   7,   9,   3],
       [ 14,   8,   6, 125,  31,  11],
       [ 23,  21,  11,  19,  49,   9],
       [ 13,  11,   1,  14,  16,  20]])